In [1]:
import cv2
import numpy as np
import ipywidgets
from IPython.display import display
import numpy as np
import base64
import time

In [2]:
print(cv2.__version__)

4.8.0-dev


In [3]:
def get_faces(img, device='cpu'):
    if device == 'cpu':
        face_cascades = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        return face_cascades.detectMultiScale(img, 1.3, 5)
    elif device == 'gpu':
        face_cascades = cv2.cuda_CascadeClassifier.create('./cascades_cuda/haarcascade_frontalface_alt.xml')
        return face_cascades.detectMultiScale(img).download()[0]

In [4]:
def blur_faces(faces, color_img, gray_img):
    for (x, y, w, h) in faces:
        cv2.rectangle(color_img,pt1=(x,y),pt2=(x+w,y+h),color=(255,0,0),thickness=2)
        roi_gray = gray_img[y:y+h, x:x+w]
        roi_color = color_img[y:y+h, x:x+w]
        roi = cv2.GaussianBlur(roi_color, (23, 23), 30)

        color_img[y:y+w, x:x+h] = roi
    return color_img

In [5]:
def face_reduction(color_img, gray_img, device='cpu'):
    faces = get_faces(gray_img, device)
    gray_img = gray_img.download() if device=='gpu' else gray_img
    color_cpu_img = color_img.download() if device=='gpu' else color_img
    blur_face = blur_faces(faces, color_cpu_img, gray_img)
    return blur_face

### Face Redaction with CPU

In [6]:
%%time
c_widget = ipywidgets.Image(format='jpeg')

img = cv2.imread('obama.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

start_time = time.time()
red_img = face_reduction(img, gray, device='cpu')
end_time = time.time()
time_taken = end_time - start_time

c_widget.value = bytes(cv2.imencode('.jpg', red_img)[1])

display(c_widget)

print(f"Time taken for face recuction in CPU: {time_taken} sec")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Time taken for face recuction in CPU: 0.6690001487731934 sec
CPU times: total: 1.19 s
Wall time: 2.37 s


### Face Reduction with GPU

In [7]:
%%time
rimage_widget = ipywidgets.Image(format='jpeg')

img_gpu = cv2.cuda_GpuMat()

img = cv2.imread('obama.jpg')

img_gpu.upload(img)

gray = cv2.cuda.cvtColor(img_gpu, cv2.COLOR_BGR2GRAY)

start_time = time.time()
red_face = face_reduction(img_gpu, gray, device='gpu')
end_time = time.time()
time_taken = end_time - start_time

rimage_widget.value = bytes(cv2.imencode('.jpg', red_face)[1])

display(rimage_widget)
print(f"Time taken for face recuction in GPU: {time_taken} sec")


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Time taken for face recuction in GPU: 0.4270615577697754 sec
CPU times: total: 1.64 s
Wall time: 13.3 s


In [21]:
ipywidgets.Widget.close_all()